### Import Numpy & Math libraries

In [1]:
import numpy as np
import math

### Assuming a 200 Layer Neural Network
#### Let's see what happens to the Mean and Std-Dev at the end of 200 layer computations when we randomly initialize from a standard distribution (Using np.random.randn)

In [2]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256)
    x = np.matmul(a, x)
print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}'.format(x.std()))

Mean : 6.803182103233671e+238
Std-Dev : inf


C:\Users\Pranav\anaconda3\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


#### Above results show that the activations blew and the mean & std-dev are too high. Let's see after which layer this actually happened...

In [3]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256)
    x = np.matmul(a, x)
    if np.isinf(x.std()):
        break
print('Layer : {}'.format(i))

Layer : 127


C:\Users\Pranav\anaconda3\lib\site-packages\numpy\core\_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


#### At Layer 127, the std-dev went close to inf!!! 

#### Let's see if we initialize it too low, having a std-dev of 0.01

In [4]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * 0.01
    x = np.matmul(a, x)
print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}'.format(x.std()))

Mean : 6.618773411677304e-161
Std-Dev : 9.306805286187753e-160


#### As we can see, the values are too low and have vanished completely!!!

#### The matrix product of our inputs x and weight matrix a that we initialized from a standard normal distribution will, on average, have a standard deviation very close to the square root of the number of input connections i.e √256

In [5]:
for i in range(200):
    x = np.random.randn(256)
    a = np.random.randn(256, 256)
    y = np.matmul(a, x)
    
print('Mean : {}'.format(y.mean()))
print('Std-Dev : {}'.format(y.std()))

print('Sqrt of 256 : {}'.format(math.sqrt(256)))

Mean : -2.033238568959367
Std-Dev : 16.671325799966347
Sqrt of 256 : 16.0


#### We see a Std-Dev of 0 when we initialize with 1 as parameter for the standard normal distribution

In [6]:
for i in range(200):
    x = np.random.randn(1)
    a = np.random.randn(1) 
    y = np.matmul(a, x)
    
print('Mean : {}'.format(y.mean()))
print('Std-Dev : {}'.format(y.std()))

Mean : -0.35479144564342663
Std-Dev : 0.0


#### If we first scale the weight matrix a by dividing all its randomly chosen values by √256, the element-wise multiplication that fills in one element of the outputs y would now, on average, have a variance of only 1/√256.

In [7]:
for i in range(200):
    x = np.random.randn(256)
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    y = np.matmul(a, x)
    
print('Mean : {}'.format(y.mean()))
print('Std-Dev : {}\n'.format(y.std()))

Mean : 0.0174401771173429
Std-Dev : 0.9460319973401099



#### Let's scale the weights by 1/√n, where n is the number of network input connections at a layer (256 in our case)

In [8]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    x = np.matmul(a, x)
    
print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))    

Mean : 0.07338313940839904
Std-Dev : 1.3934180755815289



#### Voila!!! Certainly did not explode at the end of 200 layers also!!!

#### Let's see what happens if we scale the weights and have a network size of 1000

In [9]:
x = np.random.randn(256)

for i in range(1000):
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    x = np.matmul(a, x)

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))  

Mean : -0.020828393550299015
Std-Dev : 0.717778763909457



#### Certainly impressive results, they are not yet blown even after 1000 layer

### Let's attach activation functions and look at the results

In [10]:
def tanh(x):
    return np.tanh(x)

def sigmoid(x):
    s = 1.0/(1 + np.exp(-x))
    return s

def ReLU(x):
    r = np.maximum(0, x)
    return r

#### Tanh Activation + Standard Normal Distribution

In [11]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    x = tanh(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : -0.002034847274239449
Std-Dev : 0.032827911434005104



#### Sigmoid Activation + Standard Normal Distribution

In [12]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    x = sigmoid(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : 0.5046677078887747
Std-Dev : 0.11297762639466236



#### Conceptually, it makes sense that when using activation functions that are symmetric about zero and have outputs inside [-1,1], such as softsign and tanh, we’d want the activation outputs of each layer to have a mean of 0 and a standard deviation around 1, on average. Let's see if applied to ReLU


#### ReLU +  Standard Normal Distribution fails drastically

In [13]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * math.sqrt(1./256)
    x = ReLU(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : 3.360893927132041e-31
Std-Dev : 5.391334907304023e-31



#### In ReLU the Std-dev is quite close to the square root of the number of input connections, divided by the square root of two, or √256/√2 here

In [14]:
for i in range(200):
    x = np.random.randn(256)
    a = np.random.randn(256, 256) 
    y = ReLU(np.matmul(a, x))
    
print('Mean : {}'.format(y.mean()))
print('Std-Dev : {}\n'.format(y.std()))

Mean : 6.345789744210585
Std-Dev : 9.193129641219196



In [15]:
math.sqrt(256/2)

11.313708498984761

#### Scaling the weights by this number i.e. 2 will help our cause. One more reason of choosing two is increase the influence by 2 as half of the weights are lost when f(x)<0

In [16]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.randn(256, 256) * math.sqrt(2./256)
    x = ReLU(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : 0.3403720852053421
Std-Dev : 0.5489153045498826



#### Tanh + Uniform Normal Distribution fails drastically

In [17]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.uniform(-1, 1, (256, 256)) * math.sqrt(1./256)
    x = tanh(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : -1.2651254607463406e-49
Std-Dev : 1.7536069498096755e-48



#### Xavier Initialization + Tanh works well

In [18]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.uniform(-1, 1, (256, 256)) * math.sqrt(6./(256 + 256))
    x = tanh(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : 0.0037240065404603416
Std-Dev : 0.05154912410212177



#### Xavier Initialization + ReLU fails!

In [19]:
x = np.random.randn(256)

for i in range(200):
    a = np.random.uniform(-1, 1, (256, 256)) * math.sqrt(6./(256 + 256))
    x = ReLU(np.matmul(a, x))

print('Mean : {}'.format(x.mean()))
print('Std-Dev : {}\n'.format(x.std()))

Mean : 3.0915757299320967e-31
Std-Dev : 4.1623075606803016e-31



## Summary

#### Xavier Init with ReLU fails
#### Xavier Init and Std Normal Distribution with all zero-centered activations will work well (Tanh, Sigmoid etc)
#### Std Normal Distribution scaled by sqrt(2/n) with ReLU works well where n is no. of neurons